In [ ]:
import pandas as pd

# Pfad zur Datei
tsv_path = "filtered_transcripted_train_all.tsv"
df = pd.read_csv(tsv_path, sep="\t", index_col=0)

print(df.head())
print(df.columns)

In [ ]:
tot = len(df)
uniq = df['sentence'].nunique()

print("Total Einträge in DataFrame:", tot)
print("Einzigartige Einträge in DataFrame:", uniq)

In [ ]:
audio_len = df.duration.mean()
word_sent = df['sentence_norm'].str.split().str.len().mean()

print('Durchschnittliche Audio Länge [s]:', audio_len)
print('Durchschnittliche Anzahl Wörter pro Satz:', word_sent)

## Verteilung der Confidence-Werte
Die folgenden Plots zeigen die Verteilung der Confidence-Scores und markieren Ausreißer für eine spätere Filterung.


In [ ]:
df.hist("confidence")

## Verteilung der WER-Werte
Zusätzlich zum Confidence-Blick wird die WER-Verteilung visualisiert, um problematische Segmente schneller zu identifizieren.


In [ ]:
df.hist("WER")

## Embeddings berechnen und Kennzahlen hinzufügen
Embeddings, Similarity-Scores und zusätzliche Spalten werden hier erzeugt und persistiert, sodass jeder weitere Schritt auf denselben Merkmalen aufsetzt.


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# ---------------------------------------------------
# 1. TSV einlesen
# ---------------------------------------------------
tsv_path = "filtered_transcripted_train_all.tsv"   # <--- anpassen
df = pd.read_csv(tsv_path, sep="\t", index_col=0)

# Wir verwenden:
# - clip_path
# - sentence_norm
# - transcript_norm
# - WER   (ist schon im TSV vorhanden)

print(df[["clip_path", "sentence_norm", "transcript_norm", "WER"]].head())

# ---------------------------------------------------
# 2. Sentence-Transformer laden
# ---------------------------------------------------
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

# ---------------------------------------------------
# 3. Embeddings für alle Sätze berechnen (batchweise)
# ---------------------------------------------------
sentences_ref = df["sentence_norm"].tolist()
sentences_hyp = df["transcript_norm"].tolist()

# normalize_embeddings=True -> Cosine Similarity = einfacher Dot-Product
ref_embs = model.encode(
    sentences_ref,
    convert_to_numpy=True,
    normalize_embeddings=True,
    batch_size=64,
    show_progress_bar=True
)

hyp_embs = model.encode(
    sentences_hyp,
    convert_to_numpy=True,
    normalize_embeddings=True,
    batch_size=64,
    show_progress_bar=True
)

print("ref_embs shape:", ref_embs.shape)  # (N, dim)
print("hyp_embs shape:", hyp_embs.shape)

# ---------------------------------------------------
# 4. Cosine Similarity + Quality Score (WER aus TSV)
# ---------------------------------------------------
# Cosine sim = Zeilenweises Dot-Produkt
emb_sim = np.sum(ref_embs * hyp_embs, axis=1)

df["emb_sim"] = emb_sim

alpha = 0.6  # Gewichtung Embedding vs. WER

# WER ist schon in [0,1]; wir nehmen (1 - WER) als "Word-Accuracy"
df["quality_score"] = alpha * df["emb_sim"] + (1 - alpha) * (1.0 - df["WER"].astype(float))

# Optional: kurz anschauen
print(df[[
    "sentence_norm",
    "transcript_norm",
    "WER",
    "emb_sim",
    "quality_score"
]].head())

# ---------------------------------------------------
# 5. Embeddings separat als .npz speichern
# ---------------------------------------------------
ids = df["clip_path"].astype(str).values   # eindeutige ID pro Sample
row_idx = df.index.values                  # ursprünglicher DataFrame-Index

npz_path = "whisper_embeddings_ref_hyp_paraphrase-multilingual-MiniLM-L12-v2.npz"

np.savez(
    npz_path,
    ids=ids,       # z.B. "0f19ca1d-.../....flac"
    row_idx=row_idx,
    ref=ref_embs,  # Shape (N, dim)
    hyp=hyp_embs   # Shape (N, dim)
)

print(f"Embeddings gespeichert in: {npz_path}")

# ---------------------------------------------------
# 6. TSV mit neuen Spalten speichern
# ---------------------------------------------------
out_tsv_path = "your_file_with_embscores.tsv"
df.to_csv(out_tsv_path, sep="\t")
print(f"TSV mit emb_sim und quality_score gespeichert in: {out_tsv_path}")

## "Hall of Fame" und "Hall of Shame" der Transkripte
Dieser Abschnitt stellt Beispiele mit besonders guten beziehungsweise schlechten Transkriptionen zusammen.


In [ ]:
import pandas as pd

# Wenn du den DF schon im Speicher hast, kannst du das Einlesen skippen
tsv_path = "your_file_with_embscores.tsv"   # anpassen
df = pd.read_csv(tsv_path, sep="\t", index_col=0)

# Nur zur Sicherheit:
assert "emb_sim" in df.columns, "Spalte 'emb_sim' nicht gefunden!"

# ---------------------------------------------------
# 10 beste (höchste Embedding-Similarity)
# ---------------------------------------------------
best_10 = df.sort_values("emb_sim", ascending=False).head(10).copy()

print("===== 10 BESTEN NUR NACH EMBEDDING-SIMILARITY =====")
print(best_10[[
    "clip_path",
    "sentence_norm",
    "transcript_norm",
    "emb_sim",
    "WER"          # falls du WER daneben sehen willst
]].to_string(index=False))

# Optional: als TSV speichern
best_10.to_csv("top10_embedding.tsv", sep="\t")

# ---------------------------------------------------
# 10 schlechtesten (niedrigste Embedding-Similarity)
# ---------------------------------------------------
worst_10 = df.sort_values("emb_sim", ascending=True).head(10).copy()

print("\n===== 10 SCHLECHTESTEN NUR NACH EMBEDDING-SIMILARITY =====")
print(worst_10[[
    "clip_path",
    "sentence_norm",
    "transcript_norm",
    "emb_sim",
    "WER"
]].to_string(index=False))

worst_10.to_csv("bottom10_embedding.tsv", sep="\t")

## Weitere mögliche Auswertungen
- Verhältnis männlich/weiblich nach Kanton


## Verteilung der Embedding-Similarity
Die Embedding-Scores werden nach denselben Kriterien ausgewertet wie Confidence und WER.


In [ ]:
df.hist("emb_sim")

## Beispiele nach Embedding-Score-Bereich
Zur qualitativen Analyse können Beispiele innerhalb definierter Similarity-Intervalle ausgegeben werden.


In [ ]:
# Filter nach emb_sim zwischen 0.70 und 0.75
subset = df[(df["emb_sim"] >= 0.60) & (df["emb_sim"] <= 0.65)]

# 10 zufällige Zeilen ziehen (oder weniger, falls nicht genug vorhanden)
examples = subset.sample(n=min(10, len(subset)), random_state=42)

# Nur gewünschte Spalten anzeigen
examples[["WER", "emb_sim", "sentence_norm", "transcript_norm"]]


## Historische Version (veraltet)
Die weiterhin gespeicherten Zellen dokumentieren die frühere Umsetzung und dienen lediglich als Referenz.


In [ ]:
import pandas as pd

# 1) Filter anwenden
mask = (
    (df["emb_sim"] >= 0.8) &     # emb_sim >= 0.7
    (df["WER"] <= 0.5) &         # WER <= 0.8
    (df["canton"] != "Wallis")   # Canton != Wallis
)

df_clean = df[mask].copy().reset_index(drop=True)

# 2) Basis-Stats
n_before = len(df)
n_after = len(df_clean)
n_removed = n_before - n_after

# 3) Anteil WER = 0 in gefiltertem Set
n_wer_zero = (df_clean["WER"] == 0).sum()
share_wer_zero = n_wer_zero / n_after if n_after > 0 else 0.0

print(f"Gesamt vorher: {n_before}")
print(f"Übrig nach Filter: {n_after}")
print(f"Entfernt: {n_removed} ({n_removed / n_before:.1%} der Daten)")

print(f"\nDavon WER = 0: {n_wer_zero} Beispiele")
print(f"Anteil WER = 0 im gefilterten Set: {share_wer_zero:.1%}")

# Optional: als neue Trainingsbasis speichern
df_clean.to_csv("emb_scores_clean_filtered.csv", index=False)


## Erstellung der Train-/Test-Datensätze (aktuelle Version)
Dieser Abschnitt erstellt die finalen Splits für Training und Test und knüpft an die oben erzeugten Merkmalslisten an.


In [ ]:
#!/usr/bin/env python3
"""
Datenaufbereitung für ASR-Post-Editing-Experimente.

- Liest separate TSV-Dateien für Train und Test ein
- Wendet drei Varianten an:
    1) "all":     kein Filter (nur Wallis wird ausgeschlossen)
    2) "relaxed": emb_sim >= 0.75 (falls vorhanden), WER <= 0.7, canton != "Wallis"
    3) "strict":  emb_sim >= 0.8 (falls vorhanden),  WER <= 0.5, canton != "Wallis"
- Für jede Variante:
    * Basis-Stats für Train/Test nach Filterung ausgeben
    * Gefilterte Train/Test/All-CSV schreiben (kein erneuter Split)
    * Train-Set mit harten Beispielen oversamplen und zusätzliche CSV speichern
"""

from __future__ import annotations

from pathlib import Path

import pandas as pd

# -------------------------------------------------------
# Konfiguration
# -------------------------------------------------------

TRAIN_TSV = Path("filtered_transcripted_train_all.tsv")
TEST_TSV = Path("filtered_transcripted_test.tsv")
OUTPUT_DIR = Path("data_prepared")

RANDOM_STATE = 42
OVERSAMPLE_FACTOR = 2

REQUIRED_COLUMNS = {"sentence_norm", "transcript_norm", "WER", "canton"}

CONFIGS: list[dict[str, object]] = [
    {
        "name": "all",
        "emb_min": None,
        "wer_max": None,
        "exclude_wallis": True,
        "hard_wer_low": 0.4,
        "hard_wer_high": None,  # dynamisch anhand der Daten
    },
    {
        "name": "relaxed",
        "emb_min": 0.75,
        "wer_max": 0.7,
        "exclude_wallis": True,
        "hard_wer_low": 0.4,
        "hard_wer_high": 0.7,
    },
    {
        "name": "strict",
        "emb_min": 0.8,
        "wer_max": 0.5,
        "exclude_wallis": True,
        "hard_wer_low": 0.3,
        "hard_wer_high": 0.5,
    },
]


# -------------------------------------------------------
# Hilfsfunktionen
# -------------------------------------------------------

def print_basic_stats(df: pd.DataFrame, label: str) -> None:
    """Ein paar Basis-Statistiken für einen gefilterten DataFrame ausgeben."""
    n = len(df)
    if n == 0:
        print(f"\n=== Basis-Stats: {label} ===")
        print("Keine Beispiele nach Filterung vorhanden.")
        return

    n_wer_zero = (df["WER"] == 0).sum()
    share_wer_zero = n_wer_zero / n if n > 0 else 0.0

    print(f"\n=== Basis-Stats: {label} ===")
    print(f"Anzahl Beispiele: {n}")
    print(f"Davon WER = 0: {n_wer_zero} ({share_wer_zero:.1%})")
    print(f"WER-Mean: {df['WER'].mean():.4f}")
    if "emb_sim" in df.columns:
        print(f"emb_sim-Mean: {df['emb_sim'].mean():.4f}")


def ensure_required_columns(df: pd.DataFrame, split_name: str) -> None:
    missing = REQUIRED_COLUMNS.difference(df.columns)
    if missing:
        raise ValueError(f"Fehlende Spalten in {split_name}-Split: {missing}")


def load_split(path: Path, split_name: str) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Datei nicht gefunden: {path}")

    df = pd.read_csv(path, sep="\t", index_col=0)
    df = df.reset_index(drop=True)
    ensure_required_columns(df, split_name)

    return df


def apply_filter(df: pd.DataFrame, cfg: dict[str, object], split_name: str) -> pd.DataFrame:
    df_clean = df.copy()

    df_clean["WER"] = pd.to_numeric(df_clean["WER"], errors="coerce")
    if "emb_sim" in df_clean.columns:
        df_clean["emb_sim"] = pd.to_numeric(df_clean["emb_sim"], errors="coerce")

    mask = pd.Series(True, index=df_clean.index)

    emb_min = cfg.get("emb_min")
    if emb_min is not None:
        if "emb_sim" not in df_clean.columns:
            print(
                f"WARNUNG: Spalte 'emb_sim' nicht vorhanden ({split_name}). "
                "emb_min-Filter wird übersprungen."
            )
        else:
            mask &= df_clean["emb_sim"] >= float(emb_min)

    wer_max = cfg.get("wer_max")
    if wer_max is not None:
        mask &= df_clean["WER"] <= float(wer_max)

    if cfg.get("exclude_wallis", True):
        mask &= df_clean["canton"] != "Wallis"

    df_filtered = df_clean[mask].copy().reset_index(drop=True)

    n_before = len(df_clean)
    n_after = len(df_filtered)
    removed = n_before - n_after
    removed_share = (removed / n_before) if n_before else 0.0

    print(f"\n===== Filter '{cfg['name']}' ({split_name}) =====")
    print(f"Gesamt vorher: {n_before}")
    print(f"Übrig nach Filter: {n_after}")
    print(f"Entfernt: {removed} ({removed_share:.1%} der Daten)")

    print_basic_stats(df_filtered, f"{cfg['name']} ({split_name})")

    return df_filtered


def save_filtered_sets(cfg_name: str, train_df: pd.DataFrame, test_df: pd.DataFrame) -> None:
    cfg_dir = OUTPUT_DIR / cfg_name
    cfg_dir.mkdir(parents=True, exist_ok=True)

    all_df = pd.concat([train_df, test_df], ignore_index=True)

    all_path = cfg_dir / f"emb_scores_clean_{cfg_name}_all.csv"
    train_path = cfg_dir / f"emb_scores_clean_{cfg_name}_train.csv"
    test_path = cfg_dir / f"emb_scores_clean_{cfg_name}_test.csv"

    all_df.to_csv(all_path, index=False)
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)

    print(f"Gesamtes Set gespeichert: {all_path}")
    print(f"Train-Set gespeichert:   {train_path}")
    print(f"Test-Set gespeichert:    {test_path}")


def oversample_hard_cases(
    train_df: pd.DataFrame,
    config_name: str,
    wer_low: float,
    wer_high: float,
) -> pd.DataFrame:
    cfg_dir = OUTPUT_DIR / config_name
    cfg_dir.mkdir(parents=True, exist_ok=True)

    if len(train_df) == 0:
        print(f"\n--- Oversampling '{config_name}' ---")
        print("WARNUNG: Kein Train-Set vorhanden – Überspringe Oversampling.")
        oversampled_train = train_df.copy()
    else:
        wer_high = max(wer_low, wer_high)
        hard_mask = (train_df["WER"] >= wer_low) & (train_df["WER"] <= wer_high)
        hard_df = train_df[hard_mask]

        print(f"\n--- Oversampling '{config_name}' ---")
        print(f"Harte Beispiele (WER in [{wer_low}, {wer_high}]): {len(hard_df)}")

        if len(hard_df) == 0:
            print("WARNUNG: Keine harten Beispiele gefunden – Oversampling wird übersprungen.")
            oversampled_train = train_df.copy()
        else:
            hard_oversampled = hard_df.sample(
                n=len(hard_df) * (OVERSAMPLE_FACTOR - 1),
                replace=True,
                random_state=RANDOM_STATE,
            )
            oversampled_train = (
                pd.concat([train_df, hard_oversampled], axis=0)
                .sample(frac=1.0, random_state=RANDOM_STATE)
                .reset_index(drop=True)
            )

            print(f"Train-Samples vorher: {len(train_df)}")
            print(f"Train-Samples nach Oversampling: {len(oversampled_train)}")
            print_basic_stats(oversampled_train, f"Train nach Oversampling '{config_name}'")

    os_path = cfg_dir / f"emb_scores_clean_{config_name}_train_oversampled.csv"
    oversampled_train.to_csv(os_path, index=False)
    print(f"Oversampled-Train-Set: {os_path}")

    return oversampled_train


# -------------------------------------------------------
# Main
# -------------------------------------------------------

def main() -> None:
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    train_df = load_split(TRAIN_TSV, "train")
    test_df = load_split(TEST_TSV, "test")

    print(f"Input Train geladen: {TRAIN_TSV} mit {len(train_df)} Zeilen")
    print(f"Input Test geladen:  {TEST_TSV} mit {len(test_df)} Zeilen")

    for cfg in CONFIGS:
        cfg_name = str(cfg["name"])
        print(f"\n==============================\nKonfiguration: {cfg_name}\n==============================")

        train_filtered = apply_filter(train_df, cfg, "train")
        test_filtered = apply_filter(test_df, cfg, "test")

        save_filtered_sets(cfg_name, train_filtered, test_filtered)

        wer_low = float(cfg.get("hard_wer_low", 0.4))
        wer_high_cfg = cfg.get("hard_wer_high")
        if wer_high_cfg is not None:
            wer_high = float(wer_high_cfg)
        else:
            if len(train_filtered):
                wer_high = float(train_filtered["WER"].max())
            else:
                wer_high = wer_low

        oversample_hard_cases(train_filtered, cfg_name, wer_low, wer_high)

    print("\n=== Fertig. Alle CSVs wurden in", OUTPUT_DIR, "geschrieben. ===")


main()


In [ ]:
import pandas as pd
import numpy as np

tsv_path = "your_file_with_embscores.tsv"   # anpassen
df = pd.read_csv(tsv_path, sep="\t", index_col=0)

# Falls noch nicht vorhanden: kombinierten Score berechnen
alpha = 0.6  # Gewicht Embedding
if "quality_score" not in df.columns:
    df["quality_score"] = alpha * df["emb_sim"] + (1 - alpha) * (1 - df["WER"])

# ---------- Variante 1: fixe Schwellwerte ----------
# "hoch" WER: z.B. >= 0.3
# "hoch" emb_sim: z.B. >= 0.9
mask = (df["WER"] >= 0.30) & (df["emb_sim"] >= 0.90)

candidates = df[mask].copy()

# zur Sicherheit nach WER absteigend sortieren, dann nach emb_sim absteigend
candidates = candidates.sort_values(["WER", "emb_sim"], ascending=[False, False])

top10 = candidates.head(10)

print("=== 10 Beispiele mit HOHER WER und HOHER emb_sim ===")
print(top10[[
    "clip_path",
    "sentence_norm",
    "transcript_norm",
    "WER",
    "emb_sim",
    "quality_score"
]].to_string(index=False))

In [ ]:
df_quick = df[["sentence_norm", "transcript_norm"]]
df_quick.to_csv("ref.csv")